In [53]:
import pandas as pd
import numpy as np
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, OneHotEncoder, Imputer
from sklearn.metrics import mean_squared_log_error

## read data

In [92]:
train_df = pd.read_csv('/var/data/train.csv')
test_df = pd.read_csv('/var/data/test.csv')

In [48]:
print(train_df.shape)
print(test_df.shape)

(1460, 81)
(1459, 80)


## data preprocessing

### use IDs as index

In [93]:
train_df.set_index('Id', inplace=True, verify_integrity=True)
test_df.set_index('Id', inplace=True, verify_integrity=True)

### cleaning

The target variable has to be called **class** (tpot internal constraint)

In [94]:
train_df.rename(columns={'SalePrice': 'class'}, inplace=True)

### missing values imputation

Based on [kaggle doc](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [95]:
# on train set
train_df.LotFrontage.fillna(value=0.0, inplace=True)
train_df.Alley.fillna(value='No alley', inplace=True)
train_df[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']] = train_df[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].fillna(value='No basement')
train_df.FireplaceQu.fillna(value='No fireplace', inplace=True)
train_df[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']] = train_df[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']].fillna(value='No garage')
train_df.PoolQC.fillna(value='No pool', inplace=True)
train_df.Fence.fillna(value='No fence', inplace=True)
train_df.MiscFeature.fillna(value='No extra', inplace=True)

train_df.MasVnrType.fillna(value='Unknown', inplace=True)
train_df.drop(['MasVnrArea', 'GarageYrBlt', 'MiscVal'], axis=1, inplace=True)

Fill missing value for variable _Electrical_ with a possible value

In [8]:
print(train_df.loc[train_df.Electrical.isnull(), 'Electrical'])

Id
1380    NaN
Name: Electrical, dtype: object


In [9]:
train_df.Electrical.describe()

count      1459
unique        5
top       SBrkr
freq       1334
Name: Electrical, dtype: object

In [96]:
train_df.Electrical.fillna(value='SBrkr', inplace=True)

In [97]:
# on test set
test_df.LotFrontage.fillna(value=0.0, inplace=True)
test_df.Alley.fillna(value='No alley', inplace=True)
test_df[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']] = test_df[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].fillna(value='No basement')
test_df.FireplaceQu.fillna(value='No fireplace', inplace=True)
test_df[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']] = test_df[['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']].fillna(value='No garage')
test_df.PoolQC.fillna(value='No pool', inplace=True)
test_df.Fence.fillna(value='No fence', inplace=True)
test_df.MiscFeature.fillna(value='No extra', inplace=True)

test_df.MasVnrType.fillna(value='Unknown', inplace=True)
test_df.drop(['MasVnrArea', 'GarageYrBlt', 'MiscVal'], axis=1, inplace=True)

In [98]:
test_df.Electrical.fillna(value='SBrkr', inplace=True)

In [75]:
test_df.MSZoning.describe()

count     1455
unique       5
top         RL
freq      1114
Name: MSZoning, dtype: object

In [74]:
print(test_df.loc[test_df.MSZoning.isnull(), 'MSZoning'])

Id
1916    NaN
2217    NaN
2251    NaN
2905    NaN
Name: MSZoning, dtype: object


In [101]:
test_df.MSZoning.fillna(value='RL', inplace=True)

In [79]:
test_df.Utilities.describe()

count       1457
unique         1
top       AllPub
freq        1457
Name: Utilities, dtype: object

In [102]:
test_df.Utilities.fillna(value='AllPub', inplace=True)

In [83]:
test_df.Exterior1st.describe()

count        1458
unique         13
top       VinylSd
freq          510
Name: Exterior1st, dtype: object

In [103]:
test_df.Exterior1st.fillna(value='VinylSd', inplace=True)

In [87]:
test_df.Exterior2nd.describe()

count        1458
unique         15
top       VinylSd
freq          510
Name: Exterior2nd, dtype: object

In [104]:
test_df.Exterior2nd.fillna(value='VinylSd', inplace=True)

In [108]:
test_df.KitchenQual.describe()

count     1458
unique       4
top         TA
freq       757
Name: KitchenQual, dtype: object

In [109]:
test_df.KitchenQual.fillna(value='TA', inplace=True)

In [112]:
test_df.Functional.describe()

count     1457
unique       7
top        Typ
freq      1357
Name: Functional, dtype: object

In [114]:
test_df.Functional.fillna(value='Typ', inplace=True)

In [117]:
test_df.SaleType.describe()

count     1458
unique       9
top         WD
freq      1258
Name: SaleType, dtype: object

In [118]:
test_df.SaleType.fillna(value='WD', inplace=True)

### transform categorical variables into numerical (scikit-learn requirement)

#### Label Binariser
Transform categorical variables into one-hot-encoded variables

In [119]:
lb_var_list = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 
               'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
               'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 
               'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
               'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 
               'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
lb_transfo_list = ['lb_' + var for var in lb_var_list]
lb_model_list = ['lb_' + var + '_model' for var in lb_var_list]
lb_train_list = [var + '_train' for var in lb_var_list]

lb_transfo_dict = {}
for transfo in lb_transfo_list:
    lb_transfo_dict[transfo] = LabelBinarizer()

lb_model_dict = {}
lb_train_dict = {}
lb_test_dict = {}
for i in range(len(lb_var_list)):
    lb_model_dict[lb_model_list[i]] = lb_transfo_dict[lb_transfo_list[i]].fit(train_df[lb_var_list[i]])
    lb_train_dict[lb_train_list[i]] = lb_model_dict[lb_model_list[i]].transform(train_df[lb_var_list[i]])
    # transform test set
    lb_test_dict[lb_train_list[i]] = lb_model_dict[lb_model_list[i]].transform(test_df[lb_var_list[i]])

In [12]:
len(lb_train_dict)

43

#### One Hot Encoder
Transform numerical variables into one-hot-encoded variables

In [120]:
ohe_var_list = ['MSSubClass', 'MoSold']
ohe_transfo_list = ['ohe_' + var for var in ohe_var_list]
ohe_model_list = ['ohe_' + var + '_model' for var in ohe_var_list]
ohe_train_list = [var + '_train' for var in ohe_var_list]

ohe_transfo_dict = {}
for transfo in ohe_transfo_list:
    ohe_transfo_dict[transfo] = OneHotEncoder(sparse=False)

ohe_model_dict = {}
ohe_train_dict = {}
ohe_test_dict = {}
for i in range(len(ohe_var_list)):
    ohe_model_dict[ohe_model_list[i]] = ohe_transfo_dict[ohe_transfo_list[i]].fit(train_df[ohe_var_list[i]].values.reshape(-1, 1))
    ohe_train_dict[ohe_train_list[i]] = ohe_model_dict[ohe_model_list[i]].transform(train_df[ohe_var_list[i]].values.reshape(-1, 1))
    # transform test set
    ohe_test_dict[ohe_train_list[i]] = ohe_model_dict[ohe_model_list[i]].transform(test_df[ohe_var_list[i]].values.reshape(-1, 1))
    

In [15]:
len(ohe_train_dict)

2

#### Concatenate transformed features into numpy array

In [61]:
# Untouched variables
unt_var_list = list(set(train_df.columns) - set(lb_var_list) - set(ohe_var_list) - {'class'})

In [62]:
array_unt = train_df.as_matrix(columns=unt_var_list)
array_lb = np.hstack(tuple([value[1] for value in lb_train_dict.items()]))
array_ohe = np.hstack(tuple([value[1] for value in ohe_train_dict.items()]))

In [63]:
train_array = np.hstack((array_unt, array_lb, array_ohe))

In [39]:
train_array.shape

(1460, 322)

In [121]:
# create test_array
array_unt = test_df.as_matrix(columns=unt_var_list)
array_lb = np.hstack(tuple([value[1] for value in lb_test_dict.items()]))
array_ohe = np.hstack(tuple([value[1] for value in ohe_test_dict.items()]))

test_array = np.hstack((array_unt, array_lb, array_ohe))

## tpot 

### customised scoring function

Kaggle requirement: Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price.

In [65]:
def log_rmse(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred)

### fit

In [66]:
X_train, X_test, y_train, y_test = train_test_split(train_array, train_df['class'], train_size=0.95, test_size=0.05)

tpot = TPOTRegressor(generations=100, population_size=10, verbosity=2, scoring=log_rmse, n_jobs=-1, random_state=26)
tpot.fit(X_train, y_train)

# About the warning
# https://stackoverflow.com/questions/41238769/warning-messages-when-using-python
# https://github.com/rhiever/tpot/issues/284

Optimization Progress:   2%|▏         | 20/1010 [00:26<1:25:30,  5.18s/pipeline]

Generation 1 - Current best internal CV score: 0.1257970211387605


Optimization Progress:   3%|▎         | 30/1010 [01:11<1:20:36,  4.94s/pipeline]

Generation 2 - Current best internal CV score: 0.1257970211387605


Optimization Progress:   4%|▍         | 40/1010 [01:23<57:58,  3.59s/pipeline]  

...

Generation 98 - Current best internal CV score: 6.289576230965715


Optimization Progress: 100%|█████████▉| 1009/1010 [2:00:00<00:11, 11.33s/pipeline]

Generation 99 - Current best internal CV score: 6.289576230965715


Generation 100 - Current best internal CV score: 6.289576230965715

Best pipeline: XGBRegressor(FastICA(PCA(input_matrix, iterated_power=2, svd_solver=randomized), tol=0.45), learning_rate=0.001, max_depth=7, min_child_weight=1, n_estimators=100, nthread=1, subsample=0.05)


TPOTRegressor(config_dict={'sklearn.linear_model.ElasticNetCV': {'l1_ratio': array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,  0.95,  1.  ]), 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}, 'sklearn.ensemble.ExtraT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
       crossover_rate=0.1, cv=5, disable_update_check=False,
       early_stop=None, generations=100, max_eval_time_mins=5,
       max_time_mins=None, mutation_rate=0.9, n_jobs=4, offspring_size=10,
       periodic_checkpoint_folder=None, population_size=10,
       random_state=26, scoring=None, subsample=1.0, verbosity=2,
       warm_start=False)

In [67]:
print(tpot.score(X_test, y_test))

5.99740906298


In [42]:
tpot.export('tpot_exported_pipeline.py')

True

In [43]:
tpot.evaluated_individuals_

{'DecisionTreeRegressor(DecisionTreeRegressor(input_matrix, DecisionTreeRegressor__max_depth=3, DecisionTreeRegressor__min_samples_leaf=9, DecisionTreeRegressor__min_samples_split=13), DecisionTreeRegressor__max_depth=9, DecisionTreeRegressor__min_samples_leaf=9, DecisionTreeRegressor__min_samples_split=7)': (2,
  -1656784604.539309),
 'DecisionTreeRegressor(LinearSVR(input_matrix, LinearSVR__C=0.1, LinearSVR__dual=True, LinearSVR__epsilon=0.001, LinearSVR__loss=squared_epsilon_insensitive, LinearSVR__tol=0.01), DecisionTreeRegressor__max_depth=6, DecisionTreeRegressor__min_samples_leaf=4, DecisionTreeRegressor__min_samples_split=11)': (2,
  -1206903849.5031352),
 'DecisionTreeRegressor(LinearSVR(input_matrix, LinearSVR__C=0.5, LinearSVR__dual=True, LinearSVR__epsilon=0.001, LinearSVR__loss=squared_epsilon_insensitive, LinearSVR__tol=1e-05), DecisionTreeRegressor__max_depth=10, DecisionTreeRegressor__min_samples_leaf=16, DecisionTreeRegressor__min_samples_split=3)': (2,
  -1216302454.1

In [129]:
tpot.fitted_pipeline_

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power=2, n_components=None, random_state=None,
  svd_solver='randomized', tol=0.0, whiten=False)), ('fastica', FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
    n_components=None, random_state=None, tol=0.45, w_init=None,
    whiten=...inear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.05))])

### predict on test set

In [122]:
sales_pred = tpot.predict(test_array)

Imputing missing values in feature set


In [128]:
# Create the csv file for submission
sales_pred_df = pd.DataFrame({'Id': test_df.index, 'SalePrice': sales_pred})
sales_pred_df.to_csv('/var/data/tpot_sales_pred.csv', index = False)